In [1]:
!pip install sastrawi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 3.9 MB/s eta 0:00:00


In [3]:
import pandas as pd
import re
import string
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

#Add korpus stopword tambahan yang disediakan oleh DSI(Data Science Indonesia) lewat github mereka
list_stopword   = list(pd.read_csv('https://raw.githubusercontent.com/datascienceid/stopwords-bahasa-indonesia/master/stopwords_id_satya.txt', header = None)[0])
#Add beberapa word custom yang ingin dibuang, yaitu nama - nama redaksi serta dummy word periklanan yang terambil pada scraping
to_remove = ['liputan', 'kompas', 'tempo','okezone','advertisement','scroll','to' ,'continue' ,'with', 'content' ]

def clean_text(text):
    #membuang semua kata yang mengandung http, https dan .com
    text = re.sub(r'https?://\S+|\.com', '', text)
    #Membuang semua kata yang mengandung # ataupun @, walaupun jarang ada pada data berita, namun ada baiknya pencegahan dilakukan
    text = re.sub('(@\w+|#\w+)','',text)
    #Membuang semua tag html
    text=re.sub('<.*?>', '', text)
    #Membuang semua non-alphabetic text
    text = re.sub('[^a-zA-Z]',' ',text)
    #Mengganti newline dengan spasi, sebab data berita sangat banyak newline tidak terduga
    text = re.sub("\n"," ",text)
    #Mengganti double space menjadi satu. Ini untuk merapikan teks akibat newline di atas
    text = re.sub('(s{2,})',' ',text)
    #Gabungkan terlebih dahulu text
    text=' '.join(text.split())

    #Split text tersebut
    text_split = text.split(' ')

    #Remove Stopword dan custom_word dari teks mengandalkan Sastrawi dan korpus stopword DSI
    #1. Korpus DSI
    final_text = []
    for i in range(len(text_split)):
      if (text_split[i] not in list_stopword) and (text_split[i] not in to_remove) and (type(text_split[i]) == str):
        final_text.append(text_split[i])

    final_text = ' '.join(final_text)

    #2. Sastrawi
    stopword_remover = StopWordRemoverFactory().create_stop_word_remover()
    #Stopword dicek kembali dari final text yang sudah ada
    final_text = stopword_remover.remove(final_text)


    #Finally, yang terakhir dilakukan adalah stemming, menggunakan Sastrawi dari yang ada
    stemmer = StemmerFactory().create_stemmer()
    final_text = stemmer.stem(final_text)

    return final_text



def cleaning(df,col_names):
  #Case Folding => Kecilkan semua kolom yang ada pada sebuah dataframe, nantinya untuk teks
  df[col_names] = df[col_names].str.lower()
  df[col_names] = df[col_names].apply(clean_text)

  return df


In [5]:
df = pd.read_csv("raw_data.csv")

df_clean = cleaning(df,"Teks")

In [6]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 450 entries, 0 to 449
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Teks    450 non-null    object
 1   Media   450 non-null    object
 2   Label   450 non-null    object
dtypes: object(3)
memory usage: 10.7+ KB


In [11]:
df_clean['Teks'].sample(15)

396    jakarta main ajaib usia tahun omari forson lih...
141    jakarta presiden joko widodo ingat rukun satu ...
436    seoul akun weibo milik lisa blackpink akses mu...
434    jakarta malam anugerah piala citra festival fi...
91     lionel me i hasil raih satu harga gengsi milik...
290    alex marquez yakin sang kakak marc marquez ras...
401    jakarta pakar transfer fabrizio romano beri ka...
152    datang malaysia maret one piece exhibition asi...
382    jakarta calon presiden ganjar pranowo menyicip...
424    jakarta facele love drama thailand baru tonton...
372    jakarta gubernur jawa timur khofifah indar par...
218    co jakarta eduardo camavinga panjang kontra re...
138    jakarta bakal calon presiden ganjar pranowo wa...
320    organisasi ikat dagang pasar indonesia ikappi ...
52     bakal cawapres koalisi indonesia maju kim gibr...
Name: Teks, dtype: object

In [12]:
df_clean.to_csv("clean_data.csv")